# N-Gram Language Models
In this exercise, we will use n-gram language models to predict the probability of text, and generate it.

In [35]:
from nltk.corpus import gutenberg

First, we load Jane Austen's Emma from NLTK's gutenberg corpus that we also used in a previous exercise. Tokenize and lowercase this text such that we have a list of words.

In [36]:
import nltk

raw_text = gutenberg.raw('austen-emma.txt')
# words = [w.lower() for w in nltk.word_tokenize(raw_text)]
words = [w.lower() for w in raw_text.split()]

print(len(words))

158167


Write an n-gram language model class that takes the word list and a parameter `n` as inputs, where `n` is a positive integer larger than 1 that determines the `n` of the n-gram LM. The LM should build a dictionary of n-gram counts from the word list.

In [41]:
from collections import defaultdict

class NGramLanguageModel:
    
    def __init__(self, words, n=2):
        assert n > 1, "n needs to be a positive integer > 1"
        assert n <= len(words), "n can't be larger than the number of words"
        self.n = n
        self.counts = defaultdict(int)
        for i in range(len(words) - self.n + 1):
            ngram = tuple(words[i:i+self.n])
            self.counts[ngram] += 1
            self.counts[ngram[:-1]] += 1
        self.counts[ngram[1:]] += 1  # last n-gram in the input

Now we "train" the n-gram LM by building the n-gram counts of the Emma novel. Use a low `n` (i.e. 2 or 3).

In [42]:
ngrams = NGramLanguageModel(words, 3)

Let's add a method `log_probability` to the n-gram LM class that computes the probability of an input string. Since multiplying many probabilities (<= 1) results in very small numbers that can underflow, we sum the log probabilities instead.

In [43]:
import math

def log_probability(self, input_string):
    log_prob = []
    words = [w.lower() for w in input_string.split()]
    for i in range(len(words) - self.n + 1):
        ngram = tuple(words[i:i+self.n])
        if ngram in self.counts:
            prob = self.counts[ngram] / self.counts[ngram[:-1]]
            log_prob.append(math.log(prob))
    return sum(log_prob)

NGramLanguageModel.log_probability = log_probability

Shorter texts will have higher log probability than longer texts, so we need to normalize it by the number of words in the input string.

In [44]:
# Compute the log-probability of an input string
input_string = 'There is a house in New Orleans.'
log_prob = ngrams.log_probability(input_string)

print(f"Log-probability of '{input_string}': {log_prob}")

Log-probability of 'There is a house in New Orleans.': -2.159484249353372


Lets predict the probabilities of two novels under our trained model: Jane Austen's *Sense and Sensibility* (`austen-sense.txt`) and Shakespeare's *Hamlet* (`shakespeare-hamlet.txt`).
- What do you expect will happen?
- What do you observe?

In [46]:
austen_sense = gutenberg.raw('austen-sense.txt')
shakespeare_hamlet = gutenberg.raw('shakespeare-hamlet.txt')
print(ngrams.log_probability('There is a house in New Orleans.'))

-2.159484249353372


How many n-grams are known in each input?

In [47]:
def known_ngrams(self, input_string):
    counter = 0
    words = [w.lower() for w in input_string.split()]
    for i in range(len(words) - self.n + 1):
        ngram = tuple(words[i:i+self.n])
        if ngram in self.counts:
            counter += 1
            
    return counter, len(words) - self.n + 1

NGramLanguageModel.known_ngrams = known_ngrams

In [48]:
known, total = ngrams.known_ngrams(raw_text)
print(f'Known n-grams in Emma: {known/total:.2%} ({known}/{total})')
known, total = ngrams.known_ngrams(austen_sense)
print(f'Known n-grams in Sense and Sensibility: {known/total:.2%} ({known}/{total})')
known, total = ngrams.known_ngrams(shakespeare_hamlet)
print(f'Known n-grams in Hamlet: {known/total:.2%} ({known}/{total})')

Known n-grams in Emma: 100.00% (158165/158165)
Known n-grams in Sense and Sensibility: 15.10% (17925/118673)
Known n-grams in Hamlet: 3.06% (906/29603)


Let's add a method `generate` that takes the start of a sentence ("prompt") and a number of words to generate, then continues our prompt.

In [56]:
def generate(self, prompt, num_words=10):
    words = [w.lower() for w in prompt.split()]
    for i in range(num_words):
        prefix = tuple(words[-(self.n - 1):])
        if prefix not in self.counts:
            words.append('[END]')
            break
        next_word_dist = {}
        for ngram in self.counts:
            if len(ngram) == self.n and ngram[:-1] == prefix:
                next_word_dist[ngram] = self.counts[ngram] / self.counts[prefix]
        # print(prefix, next_word_dist)
        best_ngram, prob = max(next_word_dist.items(), key=lambda x: x[1])
        print(best_ngram, prob)
        words.append(best_ngram[-1])
    return ' '.join(words)

NGramLanguageModel.generate = generate

Play around with a few different prompts.

In [66]:
ngrams.generate('I did', num_words=5)

('i', 'did', 'not') 0.5862068965517241
('did', 'not', 'know') 0.11627906976744186
('not', 'know', 'what') 0.20588235294117646
('know', 'what', 'to') 0.17647058823529413
('what', 'to', 'do.') 0.25


'i did not know what to do.'